# 2장

## 텐서 다루기

### 텐서 생성

In [ ]:
import torch

In [ ]:
print(torch.tensor([[1,2],[3,4]]))  #2차원 형태의 텐서 생성
print(torch.tensor([[1,2],[3,4]], device="cuda:0"))  #gpu에 텐서 생성
print(torch.tensor([[1,2],[3,4]],dtype=torch.float64))  #dtype을 이용하여 텐서 생성

tensor([[1, 2],
        [3, 4]])
tensor([[1, 2],
        [3, 4]], device='cuda:0')
tensor([[1., 2.],
        [3., 4.]], dtype=torch.float64)


### 텐서를 ndarray로 변환

In [ ]:
temp = torch.tensor([[1,2],[3,4]])
print(temp.numpy())  #텐서를 ndarray로 변환

[[1 2]
 [3 4]]


In [ ]:
temp = torch.tensor([[1,2],[3,4]], device="cuda:0")
print(temp.to("cpu").numpy())  #gpu상의 텐서를 cpu의 텐서로 변환한 후 ndarray로 변환

[[1 2]
 [3 4]]


### 텐서의 인덱스 조작

In [ ]:
temp = torch.FloatTensor([1,2,3,4,5,6,7])  #파이토치로 1차원 벡터 생성
print(temp[0],temp[1],temp[-1])  #인덱스로 접근
print('--------------------------')
print(temp[2:5],temp[4:-1])  #슬라이스로 접근

tensor(1.) tensor(2.) tensor(7.)
--------------------------
tensor([3., 4., 5.]) tensor([5., 6.])


### 벡터 두 개를 생성하여 사칙 연산

In [ ]:
v = torch.tensor([1,2,3])  #길이가 3인 벡터 생성
w = torch.tensor([3,4,6])
print(w-v)  #길이가 같은 벡터 간 뺄셈 연산

tensor([2, 2, 3])


### 텐서의 차원을 조작하는 코드

In [ ]:
temp = torch.tensor([
    [1,2],[3,4]])  #2*2 행렬 생성

print(temp.shape)
print('-----------------------')
print(temp.view(4,1))  #2*2 행렬을 4*1로 변형
print('-----------------------')
print(temp.view(-1))  #2*2 행렬을 1차원 벡터로 변형
print('-----------------------')
print(temp.view(1,-1))  #-1은 (1,?)와 같은 의미로 다른 차원으로부터 해당 값을 유추하겠다는 것입니다. temp의 원소 개수(2*2=4)를 유지한 채 (1,?)의 형태를 만족해야 하므로 (1,4)가 됩니다.
print('-----------------------')
print(temp.view(-1,1))  #앞에서와 마찬가지로 (?,1)의 의미로 temp의 원소 개수(2*2=4)를 유지한 채 (?,1)의 형태를 만족해야 하므로 (4,1)이 됩니다.


torch.Size([2, 2])
-----------------------
tensor([[1],
        [2],
        [3],
        [4]])
-----------------------
tensor([1, 2, 3, 4])
-----------------------
tensor([[1, 2, 3, 4]])
-----------------------
tensor([[1],
        [2],
        [3],
        [4]])


## 데이터 준비

### 필요한 라이브러리 설치

In [ ]:
pip install pandas

### 라이브러리 호출

In [ ]:
import pandas as pd  #pandas 라이브러리 호출
import torch  #torch 라이브러리 호출
data = pd.read_csv('../class2.csv')  #csv 파일을 불러옵니다.

x = torch.from_numpy(data['x'].values).unsqueeze(dim=1).float()  #csv 파일의 x칼럼의 값을 넘파이 배열로 받아 tensor(dtype)으로 바꾸어 줍니다.
y = torch.from_numpy(data['y'].values).unsqueeze(dim=1).float()  #csv 파일의 y칼럼의 값을 넘파이 배열로 받아 tensor(dtype)으로 바꾸어 줍니다.

### 커스텀 데이터셋을 만들어서 사용

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
  def __init__(self):  #필요한 변수를 선언하고, 데이터셋의 전처리를 해 주는 함수
  def __len__(self):  #데이터셋의 길이. 즉, 총 샘플의 수를 가져오는 함수
  def __getitem__(self, index):  #데이터셋에서 특정 데이터를 가져오는 함수(index번째 데이터를 변환하는 함수이며, 이때 반환되는 값은 텐서의 형태를 취해야 합니다.)

### 커스텀 데이터셋 구현 방법

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class CustomDataset(Dataset):
  def __init__(self, csv_file):  #csv_file 파라미터를 통해 데이터셋을 불러옵니다.
      self.label = pd.read_csv(csv_file)

  def __len__(self): #전체 데이터셋의 크기(size)를 반환합니다.
      return len(self.label)

  def __getitem__(self, idx):  #전체 x와 y 데이터 중에 해당 idx번째의 데이터를 가져옵니다.
      sample = torch.tensor(self.label.iloc[idx,0:3]).int()
      label = torch.tensor(self.label.iloc[idx,3]).int()
      return sample,label

tensor_dataset = CustomDataset('../covtype.csv')  #데이터셋으로 covtype.csv를 사용합니다.
dataset = DataLoader(tensor_dataset, batch_size=4, shuffle=True)  #데이터셋을 torch.utils.data.DataLoader에 파라미터로 전달합니다.

In [ ]:
for i, data in enumerate(dataset,0):
  print(i, end='')
  batch=data[0]
  print(batch.size())

### requests 라이브러리 설치

In [ ]:
pip install requests

### MNIST 데이터셋을 내려받음

In [ ]:
import torchvision.transforms as transforms

mnist_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(1.0,))
])  #평균이 0.5, 표준편차가 1.0이 되도록 데이터의 분포(normalize)를 조정

from torchvision.datasets import MNIST
import requests
download_root = '../chap02/data/MNIST_DATASET'  #내려받을 경로 지정

In [ ]:
train_dataset = MNIST(download_root, transform=mnist_transform, train=True,
                      download=True)  #훈련(trainig) 데이터셋
valid_dataset = MNIST(download_root, transform=mnist_transform, train=False,
                      download=True)  #검증(validation) 데이터셋
test_dataset = MNIST(download_root, transform=mnist_transform, train= False,
                     download=True)  #테스트(test) 데이터셋

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 15257791.25it/s]


Extracting ../chap02/data/MNIST_DATASET/MNIST/raw/train-images-idx3-ubyte.gz to ../chap02/data/MNIST_DATASET/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 480011.47it/s]


Extracting ../chap02/data/MNIST_DATASET/MNIST/raw/train-labels-idx1-ubyte.gz to ../chap02/data/MNIST_DATASET/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 3670126.74it/s]


Extracting ../chap02/data/MNIST_DATASET/MNIST/raw/t10k-images-idx3-ubyte.gz to ../chap02/data/MNIST_DATASET/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 9724619.08it/s]

Extracting ../chap02/data/MNIST_DATASET/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../chap02/data/MNIST_DATASET/MNIST/raw



## 모델 정의

### 단순 신경망을 정의하는 방법

In [ ]:
model = nn.Linear(in_features=1, out_features=1, bias=True)

### 파이토치에서 모델을 정의하는 코드

In [ ]:
class MLP(Module):
  def __init__(self, inputs):
      super(MLP, self).__init__()
      self.layer = Linear(inputs, 1)  #계층 정의
      self.activation = Sigmoid()  #활성화 함수 정의

  def forward(self, X):
      X = self.layer(X)
      X = self.activation(X)
      return X

NameError: name 'Module' is not defined

### sequential 신경망을 정의하는 방법

In [ ]:
import torch.nn as nn
class MLP(nn.Module):
  def __init__(self):
    self.layer1=nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(2))

    self.layer2=nn.Sequential(
        nn.Conv2d(in_channels=64, out_channels=30, kernel_size=5),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(2))

    self.layer3=nn.Sequential(
        nn.Linear(in_features=30*5*5, out_features=10, bias=True),
        nn.ReLU(inplace=True))

    def forward(self, x):
      x= self.layer1(x)
      x= self.layer2(x)
      x=x.view(x.shape[0],-1)
      x=self.layer3(x)
      return x
model = MLP()

print("Printing children\n-----------------")
print(list(model,children()))
print("\n\nPrinting Modules\n--------------")
print(list(model.modules()))

### 함수로 신경망을 정의하는 방법

In [ ]:
def MLP(in_features=1, hidden_features=20, out_features=1):
  hidden = nn.Linear(in_features=in, out_features=hidden_features,
                     bias=True)
  activation = nn.ReLU()
  output=nn.Linear(in_features=hidden_features, out_features=out_features,
                   bias=True)
  net = nn.Sequential(hidden, activation,output)
  return net

## 모델의 파라미터 정의

In [ ]:
from torch.optim import optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer,
                                              lr_lambda=lambda epoch: 0.95 ** epoch)
for epoch in range(1, 100+1):  #에포크 수만큼 데이터를 반복하여 처리
    for x, y in dataloader:  #배치 크기만큼 데이터를 가져와서 학습 진행
        optimizer.zero_grad()
loss_fn(model(x),y).backward()
optimizer.step()
scheduler.step()

## 모델 훈련

In [ ]:
for epoch in range(100):
  yhat = model(x_train)
  loss = criterion(yhat, y_train)
  optimizer.zero_grad()  #오차가 중첩적으로 쌓이지 않도록 초기화
  loss.backward()
  optimizer.step()

## 모델 평가

### 패키지 설치

In [ ]:
pip install torchmetrics

### 함수를 이용하여 모델을 평가

In [ ]:
import torch
import torchmetrixs

preds = torch.randn(10, 5).softmax(dim=-1)
target = torch.randint(5, (10,))

acc = torchmetrics.functional.accuracy(preds, target)  #모델을 평가하기 위해 torchmetrics.functional.accuracy 이용


### 모듈을 이용하여 모델을 평가

In [ ]:
import torch
import torchmetrics
metric = torchmetrics.Accuracy()  #모델 평가(정확도) 초기화

n_batches = 10
for i in range(n_batches):
   preds = torch.randn(10, 5).softmax(dim=-1)
   target = torch.randint(5, (10,))

   acc = metric(preds, target)
   print(f"Accuracy on batch {i}: {acc}")  #현재 배치에서 모델 평가(정확도)

acc = metric.compute()
print(f"Accuracy on all data: {acc}")  #모든 배치에서 모델 평가(정확도)

## 훈련 과정 모니터링

In [ ]:
pip install tensorboard

In [ ]:
import torch
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("../chap02/tensorboard")  #모니터링에 필요한 값들이 저장될 위치

for epoch in range(num_epochs):
  model,train()  #학습 모드로 전환(dropout=True)
  batch_loss = 0.0

  for i, (x, y) in enumerate(dataloader)
      x, y = x.to(device).float(), y.to(device).float()
      outputs = model(x)
      loss = criterion(outputs, y)
      writet.add_scalar("Loss",loss, epoch)  #스칼라 값(오차)을 기록
      optimizer,zero_grad()
      loss.backward()
      optimizer.step()

writer.close()  #SummaryWriter가 더 이상 필요하지 않으면 close()메서드 호출

### 텐서보드 실행

In [ ]:
tensorboard --logdir=../chap02/tensorboard -- port=6006

#웹 브라우저에서 http://localhost:6006을 입력하면 웹페이지 나옴

### model.eval()에 대한 사용 방법

In [ ]:
model.eval()  #검증 모드로 전환(dropout=False)
with torch.no_grad():  #1
    valid_loss = 0

    for x,y in valid_dataloader:
      outputs = model(x)
      loss = F.cross_entropy(outputs, y.long().squeeze())
      valid_loss += float(loss)
      y_hat +=[outputs]

valid_loss = valid_loss / len(valid_loader)

## 아나콘다 설치

## 가상 환경 생성 및 파이토치 설치

In [ ]:
# 가상 환경 만들기
conda creat -n torch_book python=3.9.0

# 가상 환경 확인
conda env list

# 가상 환경 활성화
activate torch_book

# 가상 환경 삭제
conda env remove -n torch_book

# 가상 환경에 커널 설치
conda install ipykernel

# 가상 환경에 커널 연결
ipython kermel install --name tf2_book --user

# 주피터 노트북 접속
jupyter notebook

## 파이토치 코드 맛보기

### 라이브러리 설치

In [ ]:
pip install matplotlib
pip install seaborn
pip install scikit-learn

### 업그레이드

In [ ]:
pip install --upgrade matplotlib --use-feature=2020-resolver
pip install --upgrade seaborn --use-feature=2020-resolver
pip install --upgrade scikit-learn --use-feature=2020-resolver

### 필요한 라이브러리 호출

In [ ]:
import torch
import torch.nn as nn
import numpy as np  #벡터 및 행렬 연산에서 매우 편리한 기능을 제공하는 파이썬라이브러리 패키지
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### 데이터 호출

In [ ]:
dataset = pd.read_csv('../chap02/data/car_evaluation.csv')
dataset.head()

### 예제 데이터셋 분포

In [ ]:
fig_size = plt.rcParams['figure.figsize']
fig.size[0]=8
fig.size[1]=6
plt.rcParams["figure.figsize"]=fig_size
dataset.output.value_counts().plot(kind='pie',autopct='%0.05f%%',
                                   colors=['lightblue','lightgreen','orange','pink'],explode=(0.05,0.05,0.05,0.05))

### 데이터를 범주형 타입으로 변환

In [ ]:
categorical_columns=['price','maint','doors','persons','lug_capacity','safety']  #예제 데이터셋 칼럼들의 목록

for category in categorical_columns:
  dataset[category]=dataset[category].astype('category')  #astype()매서드를 이용항 데이터를 범주형으로 변환

  price = dataset['price'].cat.codes.values
  maint=dataset['maint'].cat.codes.values
  doors=dataset['doors'].cat.codes.values
  person=dataset['persons'].cat.codes.values
  lug_capacity=dataset['lug_capacity'].cat.codes.values
  safety=dataset['safety'].cat.codes.values

  categorical_data=np.stack([price,maint,doors,person,lug_capacity,safety],1)
  categofical_data[:10]  #합친 넘파이 배열 중 열 개의 행을 출력하여 보여줍니다.


### np.stack과 np.concatenate

In [ ]:
a=np.array([[1,2],[3,4]])  #a.shape=(2,2)
b=np.array([[5,6],[7,8]])  #b.shape=(2,2)
c=np.array([[5,6],[7,8],[9,10]])  #c.shape=(3,2)

print(np.concatenate((a,b),axis=0))  #shape=(4,2)
print('-------------------')
print(np.stack((a,b),axis=0))  #shape=(2,2,2)

print(np.concatenate((a,c),axis=0))  #shape=(5,2)

print(np.stack((a,c),axis=0))  #차원이 달라 오류발생

### 배열을 텐서로 변환

In [ ]:
categorical_data = torch.tensor(categorical_data,dtype=torch.int64)
categorical_data[:10]

### 레이블로 사용할 칼럼을 텐서로 변환

In [ ]:
outputs=pd.get_dumies(dataset.output)
outputs=outputs.values
outputs=torch.tensor(outputs).flatten()  #1차원 텐서로 변환

print(categorical_data.shape)
print(outputs.shape)

### 성별, 몸무게, 국적이라는 칼럼을 갖는 배열 생성

In [ ]:
import pandas as pd
import numpy as np

data = {
    'gender' : ['male','female','male'],
    'weight' : [72,55,68],
    'nation' : ['Japan','Korea','Australia']
    }

df=pd.DataFrame(data)
df

#성별과 국적을 숫자로 변환
pd.get_dumies(df)

### revel(),reshpe(),flatten()

In [ ]:
a=np.array([[1,2],
            [3,4]])
print(a.revel())
print(a.reshape(-1))
print(a.flatten())

범주형 칼럼을 n차원으로 변환

In [ ]:
categorical_columns_sizes=[len(dataset[column].cat.categories) for column in categorical_columns]
categorical_embedding_sizes=[(col_size,min(50,(col_size+1)//2)) for col_size in categorical_column_sizes]
print(categorical_embedding_sizes)

### 데이터셋 분리

In [ ]:
total_records=1728
test_records=int(total_records * .2)  #전체 데이터 중 20%를 테스트 용도로 사용

categorical_train_data=categorical_data[:total_records-test_records]
categorical_test_data = categorical_data[total_records-test_records:total_records]
train_outputs=outputs[:total_records-test_records]
test_outputs=outputs[total_records-test_records:total_records]

### 데이터셋 분리 확인

In [ ]:
print(len(categorical_train_data))
print(len(train_outputs))
print(len(categorical_test_dat))
print(len(test_outputs))

### 모델의 네트워크 생성

In [ ]:
class Model(nn.Module):
  def __init__(self, embedding_size, outut_size,layer, p=0.4):
    super().__init__()
    self.all_embeddinfs=nn.ModuleList([nn.Embedding(ni,nf) for ni, nf in embedding_size])
    self.embeddinf_dropout = nn.Dropout(p)

    all_layer=[]
    num_categorical_cols=sum((nf for ni, nf in embeddinf_size))
    input_size=num_categorical_cols  #입력층의 크기를 찾기 위해 범주형 칼럼 개수를 input_size 변수에 저장

    for i in layer:
        all_layers.append(nn.Linear(input_size,i))
        all_layers.append(nn.ReLU(inplace=True))
        all_layers.append(nn.BatchNormld(i))
        all_laters.append(nn.Dropout(p))
        input_size=i

    all_layers.append(nn.Linear(layers[-1],output_size))
    self.layers=nn.Sequential(*all_layers)  #신경망의 모든 계층이 순차적으로 실행되도록 모든 계층에 대한 목록(all_layers)을 nn.Sequential클래스로 전달

    def forward(self,x_categorical):
      embeddings=[]
      for i,e in enumerate(self.all_embeddings):
        embeddings.append(e(x_categorical[:,i]))
        x=torch.cat(embeddings,1)  #넘파이의 concatenate와 같지만 대상이 텐서가 됩니다.
        x=self.embedding_dropout(x)
        x=self.layers(x)
        return x

### Model 클래스의 객체 생성

In [ ]:
model = Model(categorical_embedding_sizes,4,[200,100,50],p=0.4)
print(model)

### 모델의 파라미터 정의

In [ ]:
loss_function=nn.CrossEntropyLoss()
optimizer=torch.optim.Ada,(model.parameters(),lr=0.001)

### CPU/GPU 사용 지정

In [ ]:
if tirch.cuda.is_available():
    device=torch.device('cuda')  #GPU가 있다면 GPU사용
else:
    device=torch.device('cpu')  #GPU가 없다면 CPU를 사용

### 모델 학습

In [ ]:
epochs = 500
aggregated_losses=[]
train_outputs=train_outputs.to(devise=device,dtype=torch.int64)
for i in range(epochs):  #for 문은 500회 반복되며, 각 반복마다 손실 함수가 오차를 계산
    i +=1
    y_pred = model(categorical_train_data).to(device)
    single_loss=loss_function(y_pred,train_outputs)
    aggregated_losses.append(single_loss)  #반복할 때마다 오차를 aggregated_losses에 추가

    if i%25==1:
        print(f'epoch:{i:3} loss:{single_loss.item():10.8f}')

    optimizer.zero_grad()
    single_loss.backward()  #가중치를 업데이트하기 위해 손실함수의 backward()매서드 호출
    optimizer.step()  #옵티마이저 함수의 step() 매서드를 이용하여 기울기 업데이트

print(f'epoch: {i:3} loss: {single_loss.iten():10.10f}')  #오차가 25 에포크마다 출력

### 테스트 데이터셋으로 모델 예측

In [ ]:
test_outputs = test_outputs.to(device=device,dtype=torch.int64)
with torch.no_grad():
  y_val=model(categorical_test_data)
  loss=loss_function(y_val,test_outputs)
  print(f'Loss:{loss:.8f}')

### 모델의 예측 확인

In [ ]:
print(y_val[:5])

### 가장 큰 값을 갖는 인덱스 확인

In [ ]:
y_val = np.argmax(y_val,axis=1)
print(y_val[:5])

### 테스트 데이터셋을 이용한 정확도 확인

In [ ]:
from sklearn.metrics import classfication_report, confusion_matrix, accuracy_score
print(confusion_matrix(test_outputs,y_val))
print(classfication_report(test_outputs,y_val))
print(accuracy_score(test_outputs,y_val))